//+------------------------------------------------------------------+
//|                                               TickPOC_EA.mq5    |
//|                        Pure MQL5, MT5 only                       |
//+------------------------------------------------------------------+
#property strict
#property version   "1.00"
#property description "EA that calculates tick-volume POC and plots developing POC"

//--- input parameters
input ENUM_TIMEFRAMES TimeFrame = PERIOD_M1;  // Timeframe for analysis
input int               BarsToAnalyze = 500; // Number of bars to calculate POC

//--- global variables
double TickVolumes[];       // Tick volumes array
double PricePOC[];          // POC price per bar
datetime LastTime = 0;

//+------------------------------------------------------------------+
//| Expert initialization function                                   |
//+------------------------------------------------------------------+
int OnInit()
  {
   // Set timer for frequent updates
   EventSetTimer(1);  // 1 second update
   Print("Tick POC EA initialized.");
   return(INIT_SUCCEEDED);
  }
//+------------------------------------------------------------------+
//| Expert deinitialization function                                 |
//+------------------------------------------------------------------+
void OnDeinit(const int reason)
  {
   EventKillTimer();
   ObjectsDeleteAll(0,0,OBJ_HLINE);
  }
//+------------------------------------------------------------------+
//| Timer function                                                   |
//+------------------------------------------------------------------+
void OnTimer()
  {
   CalculatePOC();
  }
//+------------------------------------------------------------------+
//| Calculate POC function                                           |
//+------------------------------------------------------------------+
void CalculatePOC()
  {
   MqlRates rates[];
   int copied = CopyRates(_Symbol, TimeFrame, 0, BarsToAnalyze, rates);
   if(copied <= 0) return;

   // Tick-volume POC calculation
   ArrayResize(TickVolumes, copied);
   ArrayResize(PricePOC, copied);

   for(int i=0; i<copied; i++)
     {
      double maxVolume = 0;
      double pocPrice = rates[i].close; // fallback
      
      // We'll use tick-volume at 0.01 price steps
      double step = SymbolInfoDouble(_Symbol,SYMBOL_POINT)*10;
      double low = rates[i].low;
      double high = rates[i].high;
      
      int bins = int((high-low)/step)+1;
      double volumes[];
      ArrayResize(volumes,bins);
      
      // Accumulate tick volume per price bin
      double tickVolume = rates[i].tick_volume;
      double midPrice = (low+high)/2.0;
      int idx = int((midPrice-low)/step);
      if(idx>=0 && idx<bins) volumes[idx] += tickVolume;
      
      // Find max volume bin
      for(int b=0;b<bins;b++)
        {
         if(volumes[b] > maxVolume)
           {
            maxVolume = volumes[b];
            pocPrice = low + b*step;
           }
        }
      
      TickVolumes[i] = maxVolume;
      PricePOC[i] = pocPrice;
     }

   PlotPOC();
  }
//+------------------------------------------------------------------+
//| Plot POC as horizontal lines                                      |
//+------------------------------------------------------------------+
void PlotPOC()
  {
   // Remove old lines
   ObjectsDeleteAll(0,0,OBJ_HLINE);

   int bars = ArraySize(PricePOC);
   for(int i=0; i<bars; i++)
     {
      string name = "POC_"+IntegerToString(i);
      if(!ObjectCreate(0,name,OBJ_HLINE,0,0,PricePOC[i]))
         Print("Failed to create line: ",name);
      ObjectSetInteger(0,name,OBJPROP_COLOR,clrRed);
      ObjectSetInteger(0,name,OBJPROP_WIDTH,1);
      ObjectSetInteger(0,name,OBJPROP_STYLE,STYLE_SOLID);
     }
  }
//+------------------------------------------------------------------+
